In [3]:
# testing

# tests for getMarketData

# len(my_list) == len(set(my_list))  # if true, the set has only unique values

In [4]:
import requests
import string 
import pandas as pd
import time

In [5]:
# int -> response
# returns the web response from the PI API
def getMarketRespose(market_id):
    api_url = "https://www.predictit.org/api/marketdata/markets/"
    joined_url = api_url + str(market_id)
    
    response = requests.get(joined_url)  # fetch market data
    if("null" == response.text):
        print("Bad predictItID -- No market with that ID")
    return response

In [6]:
# None -> list
# returns a list of all open markets 
def getAllOpenMarketIDs():
    # commenting out so I'm not spamming the API
    # api_url = "https://www.predictit.org/api/marketdata/all/"
    # 
    # response = requests.get(api_url)  # fetch market data
    if(response.status_code != 200):
        print("Please wait...")
        time.sleep(3)
        print("Please try again now. If this issue persists, the API access may have changed.")

    xml_response = response.json()
    all_markets_df = pd.DataFrame.from_dict(xml_response)


    market_id_list = []  
    
    for market_index in range(len(all_markets_df)):
        # print(all_markets_df["markets"][market_index]["id"])
        market_id = all_markets_df["markets"][market_index]["id"]
        market_id_list.append(market_id)

    return market_id_list

In [7]:
# TODO: change to reference the all dataframe so I'm not making a ton of requests
# 2.7 kB individual market vs 453 kB for all 
# int -> dataframe
def getMarketDataFrame(market_id):
    response = getMarketRespose(market_id)  
    xml_response = response.json()
    
    market_df = pd.DataFrame.from_dict(xml_response)
    return market_df

In [8]:
# int -> print data 
def printMarketData(market_id):
    # market_df = getMarketDataFrame(market_id)  # whole market, including 
    
    try:
        print("Market:\t", market_df["shortName"][0])  # every market has a shortname
    except KeyError:
        print("That market doesn't have a key \"shortName\"")
        return
    tabspacing = "\t\t"
    print("name", tabspacing, "Last Close Price")
    
    
    for contract_index in range(0, len(market_df)):
        all_contracts = market_df["contracts"][contract_index]
        print(all_contracts["name"], end=tabspacing)
        print(all_contracts["lastClosePrice"], end="\t")
        print()  # newline


In [9]:

# int -> print data 
def getBetterMarketDF(market_id):
    market_df = getMarketDataFrame(market_id)  # whole market, including 
    
    try:
        print("Market:\t", market_df["shortName"][0])
    except KeyError:
        print("That market doesn't have a key \"shortName\"")
        return
    tabspacing = "\t\t"
    print("name", tabspacing, "Last Close Price")
    
    
    for contract_index in range(0, len(market_df)):
        all_contracts = market_df["contracts"][contract_index]
        print(all_contracts["name"], end=tabspacing)
        print(all_contracts["lastClosePrice"], end="\t")
        print()  # newline







In [10]:
# TODO: use OOP to make this look better

# give market id and list of desired parameters for a particular market, return that data
def getMarketData(market_id, lst=["name", "lastClosePrice"]):
    
    # HC 
    market_id = MKT_ID = 7040   # chancellor of germany

    market_df = getMarketDataFrame(market_id)  # whole market, including 

    try:
        print("Market:\t", market_df["shortName"][0])
    except KeyError:
        print("That market doesn't have a key \"shortName\"")


    # I should be making these as classes with objects

    # market_id       = market_df['id']  # redundant
    market_name       = market_df['name']
    market_shortName       = market_df['shortName']
    market_image       = market_df['image']
    market_url       = market_df['url']
    market_contracts       = market_df['contracts']
    market_timeStamp       = market_df['timeStamp']
    market_status       = market_df['status']

    # begin specific contract data -- i.e. the most relevant info
    contract_data = market_df["contracts"][contract_index]

    contract_id = contract_data["id"]  # note, this is not the same as the market id
    contract_dateEnd = contract_data["dateEnd"]
    contract_image =            contract_data["image"]
    contract_name =             contract_data["name"]
    contract_shortName =        contract_data["shortName"]
    contract_status =           contract_data["status"]
    contract_lastTradePrice =   contract_data["lastTradePrice"]
    contract_bestBuyYesCost =   contract_data["bestBuyYesCost"]
    contract_bestBuyNoCost =    contract_data["bestBuyNoCost"]
    contract_bestSellYesCost =  contract_data["bestSellYesCost"]
    contract_bestSellNoCost =   contract_data["bestSellNoCost"]
    contract_lastClosePrice =   contract_data["lastClosePrice"]
    contract_displayOrder  =    contract_data["displayOrder"]
    
    
    return "I need to make this as an object so I can easily reference a list"

In [11]:
market_id = MKT_ID = 7040   # chancellor of germany

    market_df = getMarketDataFrame(market_id)  # whole market, including 

    try:
        print("Market:\t", market_df["shortName"][0])
    except KeyError:
        print("That market doesn't have a key \"shortName\"")


    # I should be making these as classes with objects

    # market_id       = market_df['id']  # redundant
    market_name       = market_df['name']
    market_shortName       = market_df['shortName']
    market_image       = market_df['image']
    market_url       = market_df['url']
    market_contracts       = market_df['contracts']
    market_timeStamp       = market_df['timeStamp']
    market_status       = market_df['status']

    # begin specific contract data -- i.e. the most relevant info
    contract_data = market_df["contracts"][contract_index]


def getContractDF(market_id, contract_position):
    contract_id = contract_data["id"]  # note, this is not the same as the market id
    contract_dateEnd = contract_data["dateEnd"]
    contract_image =            contract_data["image"]
    contract_name =             contract_data["name"]
    contract_shortName =        contract_data["shortName"]
    contract_status =           contract_data["status"]
    contract_lastTradePrice =   contract_data["lastTradePrice"]
    contract_bestBuyYesCost =   contract_data["bestBuyYesCost"]
    contract_bestBuyNoCost =    contract_data["bestBuyNoCost"]
    contract_bestSellYesCost =  contract_data["bestSellYesCost"]
    contract_bestSellNoCost =   contract_data["bestSellNoCost"]
    contract_lastClosePrice =   contract_data["lastClosePrice"]
    contract_displayOrder  =    contract_data["displayOrder"]

    
    #simple_df = 


IndentationError: unexpected indent (<ipython-input-11-8b3e10f93a99>, line 3)

In [ ]:
# TODO? image grabbing function
# from PIL import Image
# url = market_df["image"][0]
# im = Image.open(requests.get(url, stream=True).raw)


In [ ]:
# None -> list
# returns a list of all open markets 
def getAllMarketID():
    xml_response = response.json()
    all_markets_df = pd.DataFrame.from_dict(xml_response)

    market_id_list = []  

    for market_index in range(len(all_markets_df)):
        # print(all_markets_df["markets"][market_index]["id"])
        market_id = all_markets_df["markets"][market_index]["id"]
        market_id_list.append(market_id)
        
    return market_id_list

In [12]:
# request data for entire market 
# None -> response
def getAllMarketsResponse():
    api_url = "https://www.predictit.org/api/marketdata/all/"
    response = requests.get(api_url)  # fetch market data
    if(response.status_code != 200):
        print("Please wait for API cooldown")
        
        for second in  list(range(10))[::-1]:
            time.sleep(1)
            print(second)
        print("Please try again now. If the issue persists, API access may have changed.")
        return None
    return response

In [ ]:

my_list = getAllMarketsList()
my_list[0]

In [78]:
# None -> list of all the open markets
def getAllMarketsList():
    all_markets_response = getAllMarketsResponse()
    return all_markets_response.json()["markets"]

def fetchIfNoMarketData(market_data=False):
    if market_data is False:
        market_data = getAllMarketsList()
        print("poop")
        print(poop)
        return market_data
    else:
        return market_data
    
    
    
def getIdIndexList( market_data=False):
    market_data = fetchIfNoMarketData(market_data)
    id_index_list = []  # 
    
    for mkt_index in range(len(market_data)):
        id_index_list.append(market_data[mkt_index]["id"])
        # print(id_index_list)
    return id_index_list

def getContractsDataFrameFromIndex( mkt_index, market_data=False):
    market_data = fetchIfNoMarketData(market_data)
    # this references the first market in the list of all markets
    market_df = pd.DataFrame.from_dict(market_data[ mkt_index ]["contracts"])
    return market_df

def getContractsDataFrame( mkt_id, market_data=False):
    market_data = fetchIfNoMarketData(market_data)
     # use the mkt_id to find the market index
    market_index_list = getIdIndexList(market_data=market_data)
    
    market_df = getContractsDataFrameFromIndex(market_index_list.index(mkt_id), market_data=market_data)
    return market_df

def concatMarketString(string):
    return "market"+string.capitalize()

# append "market" in front of each column in a dataframe if it doesn't already have the label
def labelMarket(df) -> pd.DataFrame:
    if(market_df.columns[0][0:6] == "market"):  # if marketId is already labeled, they're every column is already labeled (hopefully)
        return df
    else:
        return df.rename(concatMarketString, axis="columns")

# """
# TODO
# [ ] prepend the general market data frame with the contract specific dataframe
# [ ] look at sensible plotting options. Format data in a way that's ammenable to those options
# [ ] make csv saving job so that I don't need to worry about 
# """


In [80]:
# whole_market_data = getAllMarketsList()
getContractsDataFrame(7052, market_data=whole_market_data)
# whole_market_data = getAllMarketsList

,id,dateEnd,image,name,shortName,status,lastTradePrice,bestBuyYesCost,bestBuyNoCost,bestSellYesCost,bestSellNoCost,lastClosePrice,displayOrder
0,24780,NA,https://az620379.vo.msecnd.net/images/Contract...,Armin Laschet,Laschet,Open,0.99,NaN,0.01,0.99,NaN,0.38,0
1,24779,NA,https://az620379.vo.msecnd.net/images/Contract...,Friedrich Merz,Merz,Open,0.01,0.01,NaN,NaN,0.99,0.59,0
2,24781,NA,https://az620379.vo.msecnd.net/images/Contract...,Norbert Röttgen,Röttgen,Open,0.01,0.01,NaN,NaN,0.99,0.07,0


In [106]:
# float, float -> bool
def inSweetSpot( value, sweet_spot=0.95, threshold=0.05 ):
    if( sweet_spot - threshold <= value <= sweet_spot+threshold):
        return True
    else:
        return False

for market_id in getIdIndexList(market_data=whole_market_data):
    # print(market_id)
    # print()
    #display(getContractsDataFrame(market_id, market_data=whole_market_data))
    contract_df = getContractsDataFrame(market_id, market_data=whole_market_data)
    
    if inSweetSpot(contract_df["lastClosePrice"][0], 0.94, 0.02):
        display(contract_df.filter(items=["shortName", "lastClosePrice"]))
    

,shortName,lastClosePrice
0,Democratic,0.96
1,Republican,0.04


,shortName,lastClosePrice
0,Will Trump complete his first term?,0.95


,shortName,lastClosePrice
0,Clean sweep for Democrats in 2020?,0.96


In [112]:

import time

 def logMarketData():
    while True:
        time_localtime = time.localtime()
        time_result = time.strftime("%I:%M:%S %p", time_localtime)
        print(time_result)
 
        getAllMarketsList()
        bilken = market_df["contracts"][0]
        df_bilk = pd.DataFrame.from_dict(bilken, orient="index")

        all_m_l_df = pd.DataFrame.from_dict({"lol":all_m_l})
        mkts_filename = "all_markets " + time_result + ".csv"

        all_m_l_df.to_csv(mkts_filename, index=False)
        time.sleep(60)
 

#all_m_l = getAllMarketsList()
all_m_l

# very lazy data storage
all_m_l_df


,lol
0,"{'id': 4353, 'name': 'What will be the balance..."
1,"{'id': 4366, 'name': 'Which party will control..."
2,"{'id': 5158, 'name': 'Will Donald Trump comple..."
3,"{'id': 5665, 'name': 'Will Trump pardon Jeffre..."
4,"{'id': 5717, 'name': 'Which of these 5 Europea..."
...,...
84,"{'id': 7051, 'name': 'Will Lisa Murkowski be a..."
85,"{'id': 7052, 'name': 'Who will win the German ..."
86,"{'id': 7053, 'name': 'Who will win the 2024 Re..."
87,"{'id': 7054, 'name': 'How many Senators will v..."
